In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import math
import functools
import random
import csv
import os

## ILP for the Unbounded Multiple Knapsack Problem with Fixed Processing Times and Multiple Starting Times

In [2]:
n = 10 # number of campaign
m = 10000 # number of customers
slots = 3 # 3 time slots 
slot_length = 15

s = [] # slots
for i in range(n):
    s.append(sorted(random.sample([0,1,2], random.randint(1, 3))))
p = np.full(n, slot_length) # processing time
c = np.random.randint(low = 10, high = 40, size = n) # cost
b = np.random.randint(low = 40, high = 80, size = n) # benefit
C = np.random.randint(low = 50, high = 100, size = n) # maximum cost
    
zippedList =  list(zip(s, p, c, b, C))
campaigns = pd.DataFrame(zippedList, columns = ['starting times', 'processing time' , 'cost', 'benefit', 'maximum cost'])
campaigns.index.names = ['name']
# print("Dataframe : " , campaigns, sep='\n')

# matrix with take-rates
T = np.random.random_sample((m, n))

In [20]:
# create empty model
model = gp.Model()

# add decision variables
x = model.addMVar((m,n), vtype=GRB.BINARY, name='x') # variable x_ij
z = model.addMVar((m,n,slots), vtype=GRB.BINARY, name = 'z') # variable z_ijl
v = model.addMVar((n,slots), vtype=GRB.BINARY, name = 'v') # variable v_jl

# set objective function
model.setObjective((sum((b[j]*T[i][j] - c[j]) * x[i,j] for j in range(n) for i in range(m))), GRB.MAXIMIZE)

# add constraints
model.addConstrs((((sum(c[j]*x[i,j] for i in range(m))) <= C[j]) for j in range(n)))

model.addConstrs(((sum(z[i,j,l] for j in range(n))) <= 1) for i in range(m) for l in range(slots))

model.addConstrs((z[i,j,l] == 0) for j in range(n) for i in range(m) for l in range(slots) if l not in s[j])

model.addConstrs((((sum(v[j,l] for l in s[j])) == 1) for j in range(n)))

model.addConstrs(((v[j,l] == 0) for j in range(n) for l in range(slots) if l not in s[j]))

model.addConstrs(((z[i,j,l] >= x[i,j]+v[j,l]-1) for i in range(m) for j in range(n) for l in range(slots)))

model.addConstrs(((z[i,j,l] <= x[i,j]) for i in range(m) for j in range(n) for l in range(slots)))

model.addConstrs(((z[i,j,l] <= v[j,l]) for i in range(m) for j in range(n) for l in range(slots)))

# solve model
model.optimize()

# get attributes
if model.SolCount > 0:
    x_values = []
    v_values = []
    for k in range(m*n):
        var = model.getVarByName("x[{}]".format(k))
        x_values.append((var.varName, var.x))
    for k in range(n*3):
        var = model.getVarByName("v[{}]".format(k))
        if var.x == 1.0:
            v_values.append(var.varName)

cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)
    
# write schedule into file          
with open('{}/msfptumkp.csv'.format(path), mode = 'w') as schedule:
    writer = csv.writer(schedule, delimiter = ';')
    writer.writerow(['campaign','customer','starting time', 'processing time'])

    for value in x_values:
        if value[1] == 1.0:
            count = int(value[0][value[0].find('[')+1:value[0].find(']')])
            customer = math.floor(count/n)
            campaign = count%n
            indice = int(v_values[campaign][v_values[campaign].find('[')+1:v_values[campaign].find(']')])
            starting = (indice%slots)*slot_length
            print(campaign, customer, starting, p[campaign])

schedule.close()

# export model
# model.write('msfptumkp.lp')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1010028 rows, 400030 columns and 2580030 nonzeros
Model fingerprint: 0xd42a66a4
Variable types: 0 continuous, 400030 integer (400030 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [7e-04, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 380014 rows and 106773 columns (presolve time = 6s) ...
Presolve removed 380014 rows and 106773 columns (presolve time = 10s) ...
Presolve removed 380014 rows and 106773 columns (presolve time = 16s) ...
Presolve removed 380014 rows and 106773 columns (presolve time = 20s) ...
Presolve removed 380014 rows and 106773 columns (presolve time = 25s) ...
Presolve removed 380014 rows and 106773 columns
Presolve time: 27.90s
Presolved: 630014 rows, 293257 columns, 1706494 nonzeros